In [9]:
import datetime

import numpy as np
import pandas as pd

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_regression
import matplotlib.pyplot as plt
import os






store = pd.HDFStore("/Users/nipunbatra/Downloads/wiki-temp.h5")

a = store.keys()
def num_from_key(key):
    return int(key[1:])

def key_from_num(num):
    return "/"+str(num)


ids = map(num_from_key, a)



In [10]:
ds_path = "/Users/nipunbatra/wikienergy-2013_ipsn.h5"

In [11]:
from nilmtk import *

In [12]:
ds = DataSet(ds_path)

In [13]:
original_name_dict = {b.metadata['original_name']:b.identifier.instance for b in ds.buildings.values()}
original_name_map = pd.Series(original_name_dict)
reverse_name_map = pd.Series({v:k for k,v in original_name_dict.iteritems() })

In [14]:
original_name_map[94]

1

In [15]:
train = DataSet(ds_path)
test = DataSet(ds_path)

In [16]:
from nilmtk.disaggregate import FHMM

/Users/nipunbatra/anaconda/lib/python2.7/site-packages/hmmlearn/base.py:8: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility
  from . import _hmmc


In [17]:
def find_specific_appliance(appliance_name, appliance_instance, list_of_elecs):
    for elec_name in list_of_elecs:
        appl = elec_name.appliances[0]
        if (appl.identifier.type, appl.identifier.instance) == (appliance_name, appliance_instance):
            return elec_name

In [19]:
from create_df import read_df
df, dfc, all_homes, appliance_min, national_average = read_df()

In [24]:
run all_functions.py

In [25]:
run features.py

In [21]:
import datetime

import numpy as np
import pandas as pd

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_regression
import matplotlib.pyplot as plt
import os

#START, STOP = '2013-07-01', '2013-07-31'


from nilmtk.feature_detectors.steady_states import *



store = pd.HDFStore("/Users/nipunbatra/Downloads/wiki-temp.h5")

a = store.keys()
def num_from_key(key):
    return int(key[1:])

def key_from_num(num):
    return "/"+str(num)



hvac_homes = all_homes['hvac']






from nilmtk import *
ds_path = "/Users/nipunbatra/wikienergy-2013_ipsn.h5"

ds = DataSet(ds_path)

original_name_dict = {b.metadata['original_name']:b.identifier.instance for b in ds.buildings.values()}
original_name_map = pd.Series(original_name_dict)
reverse_name_map = pd.Series({v:k for k,v in original_name_dict.iteritems() })

train = DataSet(ds_path)
test = DataSet(ds_path)



from nilmtk.disaggregate import FHMM

def find_specific_appliance(appliance_name, appliance_instance, list_of_elecs):
    for elec_name in list_of_elecs:
        appl = elec_name.appliances[0]
        if (appl.identifier.type, appl.identifier.instance) == (appliance_name, appliance_instance):
            return elec_name



In [26]:
out = {}

months_start_stop = {
    1: ('2013-01-01', '2013-01-31'),
    2: ('2013-02-01', '2013-02-28'),
    3: ('2013-03-01', '2013-03-31'),
    4: ('2013-04-01', '2013-04-30'),
    5: ('2013-05-01', '2013-05-31'),
    6: ('2013-06-01', '2013-06-30'),
    7: ('2013-07-01', '2013-07-31'),
    8: ('2013-08-01', '2013-08-31'),
    9: ('2013-09-01', '2013-09-30'),
    10: ('2013-10-01', '2013-10-31'),
    11: ('2013-11-01', '2013-11-30'),
    12: ('2013-12-01', '2013-12-31')
}

In [27]:
train = DataSet(ds_path)
train.set_window(start="2013-07-01", end="2013-07-15")
fhmm_out = {}
hart_out = {}

In [28]:

for id_home in hvac_homes[:]:
    if id_home in fhmm_out or id_home in hart_out:
        continue
    
    fhmm_out[id_home] = {}
    hart_out[id_home] = {}



    # Train on Hart
    train_power_df = store[key_from_num(id_home)]["2013-07-01":"2013-07-15"]['use']
    train_overall_survey_df = store[key_from_num(id_home)]["2013-07-01":"2013-07-15"]
    hvac_df = train_overall_survey_df['air1']
    if 'air2' in train_overall_survey_df.columns:
        hvac_df = hvac_df + train_overall_survey_df['air2']
        hvac_circuits = 2
    else:
        hvac_circuits = 1
    """
    ss_total, tr_total = find_steady_states(pd.DataFrame({"power":train_power_df}))

    from nilmtk.disaggregate import Hart85
    from nilmtk.disaggregate.hart_85 import hart85_means_shift_cluster
    from nilmtk.disaggregate import FHMM
    
    h = Hart85()
    h.transients = deepcopy(tr_total)
    pair_df = h.pair(buffer_size=20,
      min_tolerance=100, percent_tolerance=0.035,
      large_transition=1000)
    h.centroids = hart85_means_shift_cluster(pair_df, cols=[('power','active')])
    d = (h.centroids - hvac_df[hvac_df>1000].mean()).abs()
    ac_num = d[('power','active')].argmin()
    ac_identifier_tuple = ('unknown', ac_num)
    """

    # Train FHMM
    b_id = original_name_map[id_home]
    train_elec = train.buildings[b_id].elec

    num_states_dict = {ac_elec_train: 2}
    top_5_train_elec = train_elec.submeters().select_top_k(k=3)
    fhmm = FHMM()
    fhmm.train(top_5_train_elec, num_states_dict=num_states_dict)
    ac_instance=1
    
    
    for month in range(5, 11):
        print "*"*80
        print month
        print "*"*80
        test = DataSet(ds_path)
        test.set_window(start=months_start_stop[month][0], end=months_start_stop[month][1])
        print id_home
        print "--"*80
        try:
            
            overall_survey_df = store[key_from_num(id_home)][months_start_stop[month][0]:months_start_stop[month][1]]
            power_df = overall_survey_df['use']


            

            if power_df.sum()>0 and hvac_df.sum()>0:
                pdf = pd.DataFrame({"power":power_df})

                
                #pred_df_ac_hart = h.disaggregate_chunk(pdf, {}, h.transients)[[ac_num]].squeeze()
                

                


                
                test_elec = test.buildings[b_id].elec
                test_mains = test_elec.mains()
                test_mains_df = test_mains.load().next()
                ac_elec_train = train_elec[('air conditioner', 1)]
                ac_elec_test = test_elec[('air conditioner', 1)]
                
                pred_df_fhmm = fhmm.disaggregate_chunk(test_mains_df)
                pred_df_ac_fhmm = pred_df_fhmm[[find_specific_appliance('air conditioner',
                                                                  ac_instance,
                                                                  pred_df_fhmm.columns.tolist())]]
                pred_ser_ac = pred_df_ac_fhmm.squeeze()
                pred_ser_ac.name = "FHMM"

                fhmm_out[id_home][month] = pred_ser_ac.sum()
                #hart_out[id_home][month] = pred_df_ac_hart.sum()
               





        except Exception, e:
            print e, e.message
            pass


    #pred_df = pd.DataFrame(out)


KeyError: 26

In [25]:
pd.DataFrame(fhmm_out).T.mul(.000017).dropna().to_csv("/Users/nipunbatra/git/monthly-disaggregation/fhmm_disag_new.csv")
#pd.DataFrame(hart_out).T

In [50]:
fhmm.model

GaussianHMM(algorithm='viterbi', covariance_type='full', covars_prior=0.01,
      covars_weight=1,
      init_params='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ',
      means_prior=None, means_weight=0, n_components=162, n_iter=10,
      params='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ',
      random_state=None, startprob=None, startprob_prior=None, thresh=0.01,
      transmat=None, transmat_prior=None)

In [51]:
fhmm.disaggregate_chunk(test_mains_df)

,"ElecMeter(instance=7, building=1, dataset='Dataport', appliances=[Appliance(type='sockets', instance=1)])","ElecMeter(instance=8, building=1, dataset='Dataport', appliances=[Appliance(type='fridge', instance=1)])","ElecMeter(instance=1, building=1, dataset='Dataport', appliances=[Appliance(type='dish washer', instance=1)])","ElecMeter(instance=9, building=1, dataset='Dataport', appliances=[Appliance(type='air conditioner', instance=1)])","ElecMeter(instance=5, building=1, dataset='Dataport', appliances=[Appliance(type='electric furnace', instance=1)])"
localminute,,,,,
2013-03-01 00:00:00-06:00,46,113,0,0,53
2013-03-01 00:01:00-06:00,46,113,58,0,10
2013-03-01 00:02:00-06:00,46,113,58,0,10
2013-03-01 00:03:00-06:00,46,113,58,0,10
2013-03-01 00:04:00-06:00,46,113,58,0,10
2013-03-01 00:05:00-06:00,46,113,58,0,10
2013-03-01 00:06:00-06:00,46,113,58,0,10
2013-03-01 00:07:00-06:00,46,113,58,0,10
2013-03-01 00:08:00-06:00,46,113,58,0,10


In [37]:
months_start_stop[month][1]

'2013-04-31'

In [44]:
test.set_window(start='2013-04-01' ,end='2013-04-31')

ValueError: 

In [1]:
import pandas as pd

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 23 days


In [5]:
df = pd.read_csv("/Users/nipunbatra/git/monthly-disaggregation/fhmm_disag_new.csv", index_col=0)

In [8]:
df[range(4, 10)]

,5,6,7,8,9,10
94,113.879039,182.155289,201.551677,115.104773,192.586285,107.783893
370,313.001671,481.417186,477.682626,701.049842,514.693326,341.063401
545,330.009661,484.554111,445.014270,542.304386,393.073932,331.852716
624,108.298381,309.410897,420.820193,396.916034,294.601568,112.018508
871,188.198772,394.451527,375.736295,466.903164,342.185792,148.757038
1642,850.113543,1008.160928,1012.529248,1133.742512,942.006040,571.731743
2156,222.161865,249.852366,217.587216,263.150412,204.657237,137.894667
2242,530.910306,820.071364,865.862360,906.595176,695.434980,397.752468
2470,76.068591,247.208526,236.591006,302.108462,183.721159,101.091894
2814,438.287965,847.222914,861.442292,998.171235,745.482640,325.781183
